<a href="https://colab.research.google.com/github/christianwarmuth/openhpi-kipraxis/blob/main/Woche%204/4_5_Bilderkennung_1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## 0. Installieren aller Pakete

In [ ]:
# Hier die Kaggle Credentials einfügen (ohne Anführungszeichen)

%env KAGGLE_USERNAME=openhpi
%env KAGGLE_KEY=das_ist_der_key

In [ ]:
import warnings
warnings.filterwarnings('ignore')


import os
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '3'

In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns
import keras

from keras.utils.vis_utils import plot_model

from keras.models import Sequential
from keras.layers import Dense, Conv2D , MaxPool2D , Flatten , Dropout , BatchNormalization
from keras.preprocessing.image import ImageDataGenerator
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelBinarizer
from sklearn.metrics import classification_report,confusion_matrix

warnings.simplefilter('always', category=UserWarning)

In [ ]:
import pandas as pd
import numpy as np
from matplotlib import rcParams
import string
rcParams['figure.figsize'] = 14, 10

In [ ]:
def show_accuracy_loss_plot(history, num_epochs):
    epochs = [i for i in range(num_epochs)]
    fig , ax = plt.subplots(1,2)
    train_acc = history.history['accuracy']
    train_loss = history.history['loss']
    val_acc = history.history['val_accuracy']
    val_loss = history.history['val_loss']
    fig.set_size_inches(16,9)
    
    ax[0].plot(epochs , train_acc , 'go-' , label = 'Train Accuracy')
    ax[0].plot(epochs , val_acc , 'ro-' , label = 'Test Accuracy')
    ax[0].set_title('Training & Test Accuracy')
    ax[0].legend()
    ax[0].set_xlabel("Epochs")
    ax[0].set_ylabel("Accuracy")
    
    ax[1].plot(epochs , train_loss , 'g-o' , label = 'Training Loss')
    ax[1].plot(epochs , val_loss , 'r-o' , label = 'Testing Loss')
    ax[1].set_title('Training & Test Loss')
    ax[1].legend()
    ax[1].set_xlabel("Epochs")
    ax[1].set_ylabel("Loss")
    plt.show()

# 4.5 Bilderkennung 1 
<img width=70% src="https://raw.githubusercontent.com/christianwarmuth/openhpi-kipraxis/main/images/cover_sign_language.jpg">



Datensatz: 

### Sign Language MNIST Dataset
Über 34.000 Bilder zu Gebärdensprache und dazugehörige Übersetzung der Gesten in das Alphabet in A bis Z. Bilder in 28x28 Pixeln in Graustufen. 

Quelle: kaggle.com

## Download Dataset 

### Manuell
via https://www.kaggle.com/datamunge/sign-language-mnist

### Via API

Hinzufügen der kaggle.json
Speichern als ~/.kaggle/kaggle.json auf Linux, OSX, oder anderen UNIX-basierten Betriebssystemen und unter C:\Users<Windows-username>.kaggle\kaggle.json auf Windows

Siehe https://www.kaggle.com/docs/api oder https://github.com/Kaggle/kaggle-api
        
Beispiel:
~/kaggle/kaggle.json

{"username":"openHPI","key":"das_ist_der_key"}

In [ ]:
!pip3 install kaggle
!kaggle datasets download -d datamunge/sign-language-mnist

In [ ]:
import zipfile
with zipfile.ZipFile("sign-language-mnist.zip", 'r') as zip_ref:
    zip_ref.extractall("")

## Was wir erreichen wollen

Ziel dieser Einheit ist es, Bilder von amerikanischer Gebärdensprache in das Alphabet von A bis Z zu übersetzen. Hierbei haben wir einen Trainings-Datensatz mit Labels für die verschiedenen Bilder. 

<img width=70% src="https://raw.githubusercontent.com/christianwarmuth/openhpi-kipraxis/main/images/sign_color.png">

# Einlesen der Daten

In [ ]:
df_train = pd.read_csv("sign_mnist_train.csv")
df_test = pd.read_csv("sign_mnist_test.csv")

Nachdem wir die Dateien eingelesen haben, speichern wir jeweils die Labels in eigenen Variablen und entfernen sie vom ursprünglichen Datensatz. 

In [ ]:
y_test = df_test["label"]
y_train = df_train["label"]
del df_train['label']
del df_test['label']

Wir normalisieren zunächst alle Pixel-Werte für die Bilder und konvertieren die einzelnen Pixel-Reihen zu 3D Bildern. Die dritte Dimension hier ist der Farbwert (hier nur eine Graustufe).

In [ ]:
# Normalisierung
X_train = df_train.values/255
X_test = df_test.values/255

# Reshaping von 1D zu 3D
X_train = X_train.reshape(-1,28,28,1)
X_test = X_test.reshape(-1,28,28,1)

Anschließend erstellen wir erneut eine Liste mit den verschiedenen Buchstaben, in die wir die einzelnen Bilder übersetzen wollen. Im Datensatz sind nur Nummern gegeben, aber mit dieser Liste können wir die jeweilige Nummer zu ihrem Buchstaben übersetzen. 

In [ ]:
alphabetic_label = ["A", "B", "C", "D", "E", "F", "G", "H", "I", "J", "K", "L", "M", "N", "O", "P", "Q", "R", "S", "T", "U", "V", "W", "X", "Y", "Z"]

## Label Encoding

Die Labels wollen wir allerdings noch mit einem One-Hot-Encoder verarbeiten. Für jedes Label wird eine eigene Spalte eingeführt und nur eine "1" eingefügt, wenn ein Bild diesem Label angehört. 

In [ ]:
lb=LabelBinarizer()
lb.fit(y_train)
y_test_oh = lb.transform(y_test)
y_train_oh = lb.transform(y_train)

In [ ]:
# Ursprüngliches Label
y_train[0]

In [ ]:
# Label nach One-Hot-Encoding
y_train_oh[0]

# Einfaches Modell

Wir definieren zunächst ein einfaches Convolutional Neural Network mit einem Convoluational Layer, einem Pooling Layer und einem Dense Layer (Fully-Connected Layer) vor dem Output-Layer. Damit wir etwas genauer verstehen, wie dieses Modell aufgebaut ist, so geben wir uns anschließend die Form des Modelles einmal graphisch aus. 

Wir sehen hier den Aufbau des Modells sowie die Anzahl der Parameter, die trainiert werden. 

In [ ]:
model=Sequential()
model.add(Conv2D(128,kernel_size=(5,5),
                 strides=1,padding='same',activation='relu',input_shape=(28,28,1)))
model.add(MaxPool2D(pool_size=(3,3),strides=2,padding='same'))         
model.add(Flatten())

model.add(Dense(units=256,activation='relu'))
model.add(Dropout(rate=0.3))
model.add(Dense(units=24,activation='softmax'))
model.summary()

model.compile(optimizer='adam',loss='categorical_crossentropy',metrics=['accuracy'])

In [ ]:
plot_model(model, show_shapes=True, show_layer_names=True)

Zwar haben wir jetzt das Modell definiert, allerdings haben wir noch nichts trainiert. Das wollen wir im nächsten Schritt machen. Wir trainieren für 20 Epochen (also 20 Läufe durch den Datensatz). Im Laufe des Trainings speichern wir uns jeweils die Performance auf den Trainings- und Testdaten, um diese anschließend in einem Chart ausgeben zu können. 

In [ ]:
history = model.fit(X_train,y_train_oh,
         epochs = 20,
          shuffle=1,
           validation_data=(X_test,y_test_oh),
         )

Wir wollen uns nun einmal unser Training über die Zeit ansehen. Wie wir sehen, erreichen wir eine Accuracy von circa 92%. Optimal ist das noch nicht, aber durchaus ein sehr guter Anfang. Im Folgenden wollen wir noch einmal sehen, ob wir mit einem komplexeren Modell bessere Ergebnisse erzielen.  

In [ ]:
show_accuracy_loss_plot(history, num_epochs=20)

# Komplexeres Modell

Nun versuchen wir eine komplexere Modell-Architektur. Wir fügen noch jeweils zwei Convolutional Layer und Pooling Layer hinzu. Wir werden auch noch einen weiteren Dense-Layer hinzufügen. 

In [ ]:
model=Sequential()
model.add(Conv2D(128,kernel_size=(5,5),
                 strides=1,padding='same',activation='relu',input_shape=(28,28,1)))
model.add(MaxPool2D(pool_size=(3,3),strides=2,padding='same'))
model.add(Conv2D(64,kernel_size=(2,2),
                strides=1,activation='relu',padding='same'))
model.add(MaxPool2D((2,2),2,padding='same'))
model.add(Conv2D(32,kernel_size=(2,2),
                strides=1,activation='relu',padding='same'))
model.add(MaxPool2D((2,2),2,padding='same'))
          
model.add(Flatten())

model.add(Dense(units=256,activation='relu'))
model.add(Dropout(rate=0.3))
model.add(Dense(units=128,activation='relu'))
model.add(Dropout(rate=0.3))
model.add(Dense(units=24,activation='softmax'))
model.summary()

model.compile(optimizer='adam',loss='categorical_crossentropy',metrics=['accuracy'])

In [ ]:
plot_model(model, show_shapes=True, show_layer_names=True)

In [ ]:
history = model.fit(X_train,y_train_oh,
         epochs = 20,
          shuffle=1,
           validation_data=(X_test,y_test_oh),
         )

In [ ]:
show_accuracy_loss_plot(history, num_epochs=20)

Im Vergleich zum einfachen Modell haben wir nur eine minimale Verbesserung gesehen auf circa 94%. In der nächsten Einheit werden wir noch eine Strategie anwenden, um bessere Klassifikationsergebnisse mit unserem Convolutional Neural Network zu erreichen. 